In [155]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/Paolo/Desktop/Computer Science (Python)/Progetto/tmdb_5000_movies.csv')
dfcr = pd.read_csv('C:/Users/Paolo/Desktop/Computer Science (Python)/Progetto/tmdb_5000_credits.csv')

In [30]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [88]:
dfcr.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [44]:
#Operazione preliminare: creo la tabella actors, che conterrà la lista di tutti gli attori (anche ripetuti) 
#con le loro info personali e SOPRATTUTTO con l'id ed il titolo di ogni film dentro cui sono comparsi
#Quindi molti punti si faranno con delle semplici groupby su questa tabella

#ATTENZIONE! OPERAZIONE LENTA, ESEGUIRE SOLO UNA VOLTA
actors = pd.read_json(dfcr['cast'][0])
actors['movie_id'] = dfcr['movie_id'][0]
for i in np.array(range(1,len(dfcr))):
    temp = pd.read_json(dfcr['cast'][i])
    temp['movie_id'] = dfcr['movie_id'][i]
    actors = actors.append(temp)
actors = actors.merge(right = dfcr[['movie_id', 'title']], on='movie_id')

In [111]:
actors.head()

,cast_id,character,credit_id,gender,id,movie_id,name,order,title
0,242.0,Jake Sully,5602a8a7c3a3685532001c9a,2.0,65731.0,19995,Sam Worthington,0.0,Avatar
1,3.0,Neytiri,52fe48009251416c750ac9cb,1.0,8691.0,19995,Zoe Saldana,1.0,Avatar
2,25.0,Dr. Grace Augustine,52fe48009251416c750aca39,1.0,10205.0,19995,Sigourney Weaver,2.0,Avatar
3,4.0,Col. Quaritch,52fe48009251416c750ac9cf,2.0,32747.0,19995,Stephen Lang,3.0,Avatar
4,5.0,Trudy Chacon,52fe48009251416c750ac9d3,1.0,17647.0,19995,Michelle Rodriguez,4.0,Avatar


In [118]:
#1.For each movie, compute the number of cast members
actor_each_movie = actors.groupby(by = ['movie_id','title']).count()['order']
actor_each_movie = actor_each_movie.to_frame().reset_index()
actor_each_movie.head(10)

,movie_id,title,order
0,5,Four Rooms,24
1,11,Star Wars,106
2,12,Finding Nemo,24
3,13,Forrest Gump,66
4,14,American Beauty,40
5,16,Dancer in the Dark,29
6,18,The Fifth Element,115
7,19,Metropolis,21
8,20,My Life Without Me,12
9,22,Pirates of the Caribbean: The Curse of the Bla...,70


In [156]:
# 2. How many movies do not have a homepage?
len(df[df['homepage'].isnull()].dropna(subset = ['release_date']))


3090

In [133]:
def extract_year(timestamp):
    return timestamp.year


In [160]:
#3. For each year, how many movies do not have a homepage?
df2 = df.dropna(subset = ['release_date'])
df2['release_year'] = df2['release_date'].apply(pd.to_datetime)
df2['release_year'] = df2['release_year'].apply(extract_year)

df2[df2['homepage'].isnull()].groupby(by='release_year').count()['id']

C:\Users\Paolo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Paolo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


release_year
1916      1
1925      1
1927      1
1929      2
1930      1
1932      1
1933      2
1934      1
1935      1
1936      2
1937      1
1938      2
1939      2
1940      4
1941      1
1942      1
1944      2
1945      4
1946      3
1947      3
1948      3
1949      2
1950      1
1951      3
1952      4
1953      4
1954      4
1955      2
1956      3
1957      2
       ... 
1987     29
1988     28
1989     28
1990     24
1991     29
1992     29
1993     44
1994     50
1995     64
1996     86
1997    102
1998    119
1999    145
2000    145
2001    159
2002    164
2003    131
2004    149
2005    149
2006    144
2007     92
2008     97
2009     85
2010     54
2011     43
2012     92
2013    127
2014    157
2015    110
2016     31
Name: id, Length: 89, dtype: int64